In [1]:
import tensorflow as tf
import numpy as np

n_example = 5000 #生成数据的个数
n_train = 4500
n_test = 500

n_dim = 2 # 输入数据的维数
#即显模式
tf.compat.v1.enable_eager_execution()# 动态图机制
#tf.compat.v1.disable_eager_execution()


In [2]:
# 生成训练集和测试集 输入数据x,y
# 训练集输入数据 产生特征，以均匀分布随机生成 数据点(𝑥, 𝑦)
train_X = tf.random.uniform((n_train, n_dim), -10,10)

# 测试集输入数据
test_X = tf.random.uniform((n_test, n_dim), -10,10)

# 设置真实的 weight, bias
w_real = [1, 1 ,1]
b_real = 0
#生成标准输出数据
labels_train = w_real[0] * train_X[:,0] *  train_X[:,0] + w_real[1] * train_X[:,1]* train_X[:,1]  + w_real[2] * train_X[:,1] * train_X[:,0] + b_real 
labels_test = w_real[0] * test_X[:,0] *  test_X[:,0] + w_real[1] * test_X[:,1]* test_X[:,1]  + w_real[2] * test_X[:,1] * test_X[:,0] + b_real 

# 为标准输出数据添加噪声得到训练与测试输出数据
train_z = labels_train + tf.random.normal(labels_train.shape, stddev=0.01)
test_z = labels_test + tf.random.normal(labels_test.shape, stddev=0.01)

In [3]:
#train_z

In [4]:
#test_z

In [5]:
# 定义模型类
class Model():
    def __init__(self):
        # 初始化模型参数
        self.w = tf.Variable(tf.random.normal([n_dim, 10], mean=1.0 , stddev=0.1))
        self.b = tf.Variable(tf.zeros(1,))
        
        self.w1 = tf.Variable(tf.random.normal([10, 1], mean=1.0 , stddev=0.1))
        self.b1 = tf.Variable(tf.zeros(1,))
        
#         self.w2 = tf.Variable(tf.random.normal([3, 1], mean=1.0 , stddev=0.1))
#         self.b2 = tf.Variable(tf.zeros(1,))
        # 变量通过 tf.Variable 类进行创建和跟踪。tf.Variable 表示张量，对它执行运算可以改变其值。利用特定运算可以读取和修改此张量的值。
        # 更高级的库（如 tf.keras）使用 tf.Variable 来存储模型参数。
    def __call__(self,X):#该方法的功能类似于在类中重载 () 运算符，使得类实例对象可以像调用普通函数那样，以“对象名()”的形式使用。
        # 正向传递
        # 第一层
        H1 = tf.nn.relu(tf.matmul(X, self.w) + self.b)
#         # 第二层
#         H2 = tf.nn.relu(tf.matmul(H1, self.w1) + self.b1)
        # 第三层
        z = tf.nn.relu(tf.matmul(H1, self.w1) + self.b1)
        return z

In [6]:
# 实例化模型
model = Model()
# saver = tf.compat.v1.train.Saver()
# sess = tf.compat.v1.Session()
# sess.run(tf.compat.v1.global_variables_initializer())

In [7]:
# 定义损失函数MSE
# 通常情况下取回归模型的 Loss 函数为 MSE
def squared_loss(y, y_hat, n):
  y_observed = tf.reshape(y, y_hat.shape)
  return tf.matmul(tf.transpose(y_observed - y_hat), 
                   y_observed - y_hat) / 2 / n


In [8]:
# 设置数据迭代函数
def data_iter(features, labels, mini_batch):
#   Args:
#   - features: 特征矩阵 nxd 维
#   - labels: 样本，nx1 维
#   - mini_batch: 每次抽取的样本数
  features = np.array(features)
  labels = np.array(labels)
  indeces = list(range(len(features)))
  random.shuffle(indeces)
  for i in range(0, len(indeces), mini_batch):
    j = np.array(indeces[i:min(i+mini_batch, len(features))])
    yield features[j], labels[j]

In [9]:
# 计算梯度，并更新模型参数
def sgd(params, lr):
#   Args:
#   params: 模型参数
#   lr: 学习率 learning rate
  for param in params:
    param.assign_sub(lr * t.gradient(l, param))

In [10]:
# 设置学习率，迭代次数,batch大小
learning_rate = 0.001
epoch = 40
batch = 16

In [11]:
import wandb #数据可视化
import time
wandb.init(project='ex1', name=time.strftime('%m%d%H%M%S'))

wandb: Currently logged in as: jyjy2001lfx (jnjy). Use `wandb login --relogin` to force relogin


In [12]:
import random
# 开始训练
for i in range(epoch):
    for X, y in data_iter(train_X, train_z, batch):
            # 在内存中记录梯度过程
        with tf.GradientTape(persistent=True) as t:
            t.watch([model.w, model.b,model.w1, model.b1])
            # 计算本次小批量的 loss
            l = squared_loss(y, model(X), batch)
        # 计算梯度，更新参数
        sgd([model.w, model.b], learning_rate)
    # 计算本次迭代的总误差

    train_loss = squared_loss(train_z, model(train_X), len(train_X))
    wandb.log({'epoch': i, 'train_loss': tf.reduce_mean(train_loss)})
    print('epoch %d, train_loss %f' % (i + 1, tf.reduce_mean(train_loss)))
    test_loss = squared_loss(test_z, model(test_X), len(test_X))
    wandb.log({'epoch': i, 'test_loss': tf.reduce_mean(test_loss)})
    print('epoch %d, test_loss %f' % (i + 1, tf.reduce_mean(test_loss)))

test_loss = squared_loss(test_z, model(test_X), len(test_X))
print('Final Test loss %f' % (tf.reduce_mean(test_loss)))

epoch 1, train_loss 118.958725
epoch 1, test_loss 110.895454
epoch 2, train_loss 80.526779
epoch 2, test_loss 77.380219
epoch 3, train_loss 66.999260
epoch 3, test_loss 66.838142
epoch 4, train_loss 56.878422
epoch 4, test_loss 55.487015
epoch 5, train_loss 52.925468
epoch 5, test_loss 52.357197
epoch 6, train_loss 49.040100
epoch 6, test_loss 46.843433
epoch 7, train_loss 46.353794
epoch 7, test_loss 42.455090
epoch 8, train_loss 44.966373
epoch 8, test_loss 42.781700
epoch 9, train_loss 43.740887
epoch 9, test_loss 40.272388
epoch 10, train_loss 42.445038
epoch 10, test_loss 39.917042
epoch 11, train_loss 41.118366
epoch 11, test_loss 36.622387
epoch 12, train_loss 39.459991
epoch 12, test_loss 35.984268
epoch 13, train_loss 38.640076
epoch 13, test_loss 37.154236
epoch 14, train_loss 36.348103
epoch 14, test_loss 33.151711
epoch 15, train_loss 35.002457
epoch 15, test_loss 32.614517
epoch 16, train_loss 33.957405
epoch 16, test_loss 31.780867
epoch 17, train_loss 33.252586
epoch 17,

In [13]:
model.w

<tf.Variable 'Variable:0' shape=(2, 10) dtype=float32, numpy=
array([[ 0.6095568 , -9.03813   ,  1.418492  ,  2.7814357 ,  1.717465  ,
        -3.0528345 ,  1.3593342 ,  1.3406638 ,  5.352766  ,  7.724582  ],
       [ 2.3965478 , -3.411656  ,  8.182209  ,  0.5449532 ,  1.3374546 ,
        -9.155383  ,  1.6765924 ,  3.5384512 ,  3.2178736 ,  0.73620814]],
      dtype=float32)>

In [14]:
# 保存模型
# tf.keras.models.save_model(model,'./CV_ex1_model_save/one_hide_layer')
# tf.saved_model.save(model,'./CV_ex1_model_save/one_hide_layer')